This notebook deals with extracting the location polygons and converting them to a shapefile in order to use Tableau for analysing the data

In [2]:
# https://github.com/geopandas/geopandas/issues/830
import os
# os.environ["PROJ_LIB"] = "C:\Anaconda\envs\env_name\Library\share" #windows
os.environ["PROJ_LIB"] = "C:\\Users\\flavi\\Anaconda3\\Library\\share" #windows

# Imports
from area import area
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
from lxml import objectify

In [3]:
mainXml = objectify.parse('Dataset/Original/London_division_2500-150.xml')
lxmlRoot = mainXml.getroot()
allPlaces = []
POLYGON = None
for location in lxmlRoot.iterchildren():
    place = {}
    place['id'] = int(location.id)
    place['name'] = str(location.name)
    
    shape_points = []
    for point in location.shape.iterchildren():
        shape_points.append((point.xCoord, point.yCoord))
    polygon_geom = Polygon(shape_points)
    place['polygon'] = polygon_geom
    POLYGON=polygon_geom
#     print('area: {}'.format(area(polygon_geom)))
    
    allPlaces.append(place)

myDf = pd.DataFrame.from_records(allPlaces)
myDf.head()

,id,name,polygon
0,1,Trafalgar Square,"POLYGON ((-0.1304619 51.50952, -0.13048266 51...."
1,2,Big Ben,"POLYGON ((-0.12498988 51.503284, -0.12284501 5..."
2,3,Parliament Square,"POLYGON ((-0.12562872 51.502274, -0.12844071 5..."
3,4,London Eye,"POLYGON ((-0.12235283 51.503895, -0.12092333 5..."
4,5,Tate Modern,"POLYGON ((-0.0996684 51.51117, -0.101172544 51..."


In [4]:
# import pyproj    
# import shapely
# import shapely.ops as ops
# from shapely.geometry.polygon import Polygon
# from functools import partial


# # geom = Polygon([(0, 0), (0, 10), (10, 10), (10, 0), (0, 0)])
# geom_area = ops.transform(
#     partial(
#         pyproj.transform,
#         pyproj.Proj(init='EPSG:4326'),
#         pyproj.Proj(
#             proj='aea',
#             lat1=POLYGON.bounds[1],
#             lat2=POLYGON.bounds[3])),
#     geom)

# # Print the area in m^2
# print(geom_area.area)

NameError: name 'geom' is not defined

Convert data frame to a Geo Data Frame, in order to export to a shapefile

In [7]:
# import os
# os.environ["PROJ_LIB"] = "C:\Anaconda\envs\env_name\Library\share" #windows

gdf = gpd.GeoDataFrame(myDf, geometry='polygon')
gdf.crs = {'init' :'epsg:4326'} # lat long projection
gdf['area'] = gdf['polygon'].to_crs({'init': 'epsg:3395'}).map(lambda p: p.area / 10**6)
# gdf['geometry'].to_crs({'init': 'epsg:3395'})\
#                .map(lambda p: p.area / 10**6)
# gdf.to_crs({'init': 'epsg:3395'})
gdf.head()

,id,name,polygon,area
0,1,Trafalgar Square,"POLYGON ((-0.1304619 51.50952, -0.13048266 51....",0.277425
1,2,Big Ben,"POLYGON ((-0.12498988 51.503284, -0.12284501 5...",0.399994
2,3,Parliament Square,"POLYGON ((-0.12562872 51.502274, -0.12844071 5...",0.236767
3,4,London Eye,"POLYGON ((-0.12235283 51.503895, -0.12092333 5...",0.425408
4,5,Tate Modern,"POLYGON ((-0.0996684 51.51117, -0.101172544 51...",0.686323


AssertionError: 

Export GeoDataFrame to a shapefile, which is now able to be used in Tableau

In [ ]:
gdf.to_file('Dataset/Processed/London_division.shp', driver="ESRI Shapefile")